<a href="https://colab.research.google.com/github/stepsbtw/Algoritmos/blob/main/algorithms_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1) Leitura e Filtragem inicial dos dados
Carregue os dados em sua linguagem de programação preferida (por exemplo, Python com pandas).

In [133]:
import pandas as pd
import requests
from io import BytesIO

r = requests.get("https://data.brasil.io/dataset/covid19/caso.csv.gz")
caso = pd.read_csv(BytesIO(r.content), compression="gzip")

r = requests.get("https://data.brasil.io/dataset/covid19/caso_full.csv.gz")
caso_full = pd.read_csv(BytesIO(r.content), compression="gzip")

In [134]:
caso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838003 entries, 0 to 2838002
Data columns (total 13 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   date                            object 
 1   state                           object 
 2   city                            object 
 3   place_type                      object 
 4   confirmed                       int64  
 5   deaths                          int64  
 6   order_for_place                 int64  
 7   is_last                         bool   
 8   estimated_population_2019       float64
 9   estimated_population            float64
 10  city_ibge_code                  float64
 11  confirmed_per_100k_inhabitants  float64
 12  death_rate                      float64
dtypes: bool(1), float64(5), int64(3), object(4)
memory usage: 262.5+ MB


In [135]:
caso_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3853648 entries, 0 to 3853647
Data columns (total 18 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   city                                           object 
 1   city_ibge_code                                 float64
 2   date                                           object 
 3   epidemiological_week                           int64  
 4   estimated_population                           float64
 5   estimated_population_2019                      float64
 6   is_last                                        bool   
 7   is_repeated                                    bool   
 8   last_available_confirmed                       int64  
 9   last_available_confirmed_per_100k_inhabitants  float64
 10  last_available_date                            object 
 11  last_available_death_rate                      float64
 12  last_available_deaths                     

Descrição das colunas:

- **city: nome do município (pode estar em branco quando o registro é referente ao estado, pode ser preenchido com Importados/Indefinidos também).**
- city_ibge_code: código IBGE do local.
- **date: data de coleta dos dados no formato YYYY-MM-DD.**
- epidemiological_week: número da semana epidemiológica no formato YYYYWW.
- estimated_population: população estimada para esse município/estado em 2020, segundo o IBGE. (acesse o script que faz o download e conversão dos dados de população).
- estimated_population_2019: população estimada para esse município/estado em 2019, segundo o IBGE. ATENÇÃO: essa coluna possui valores desatualizados, prefira usar a coluna estimated_population.
- is_last: campo pré-computado que diz se esse registro é o mais novo para esse local, pode ser True ou False (caso filtre por esse campo, use is_last=True ou is_last=False, não use o valor em minúsculas).
- is_repeated: campo pré-computado que diz se as informações nesse registro foram publicadas pela Secretaria Estadual de Saúde no dia date ou se o dado é repetido do último dia em que o dado está disponível (igual ou anterior a date). Isso ocorre pois nem todas as secretarias publicam boletins todos os dias. Veja também o campo last_available_date.
- **last_available_confirmed: número de casos confirmados do último dia disponível igual ou anterior à data date.**
- **last_available_confirmed_per_100k_inhabitants: número de casos confirmados por 100.000 habitantes (baseado em estimated_population) do último dia disponível igual ou anterior à data date.**
- last_available_date: data da qual o dado se refere.
- **last_available_death_rate: taxa de mortalidade (mortes / confirmados) do último dia disponível igual ou anterior à data date.**
- **last_available_deaths: número de mortes do último dia disponível igual ou anterior à data date.**
- order_for_place: número que identifica a ordem do registro para este local. O registro referente ao primeiro boletim em que esse local aparecer será contabilizado como 1 e os demais boletins incrementarão esse valor.
- place_type: tipo de local que esse registro descreve, pode ser city ou state.
- **state: sigla da unidade federativa, exemplo: SP.**
- new_confirmed: número de novos casos confirmados desde o último dia (note que caso is_repeated seja True, esse valor sempre será 0 e que esse valor pode ser negativo caso a SES remaneje os casos desse município para outro).
- new_deaths: número de novos óbitos desde o último dia (note que caso is_repeated seja True, esse valor sempre será 0 e que esse valor pode ser negativo caso a SES remaneje os casos desse município para outro).

Recomendadas para análise:
- city
- state
- last_avaible_confirmed
- last_available_deaths
- estimated_population
- last_available_confirmed_per_100k_inhabitants
- last_available_death_rate
- date

Para análises estatísticas mais simples:

In [136]:
# Uma linha por município
df = caso[(caso["is_last"]==True) & (caso["place_type"]=="city")]

Para análise de desempenho de algoritmos com volumes maiores de dados (várias datas por município):

In [137]:
# Séries temporais completas.
df_full = caso_full[(caso_full["place_type"]=="city") & (caso_full["is_repeated"]==False)]

Checar e tratar (remover linhas) colunas com valores nulos.

In [138]:
df.count()

,0
date,5589
state,5589
city,5589
place_type,5589
confirmed,5589
deaths,5589
order_for_place,5589
is_last,5589
estimated_population_2019,5570
estimated_population,5570


In [139]:
df_clean = df.dropna(subset=["confirmed_per_100k_inhabitants", "estimated_population"])
df_full.count()

,0
city,2818241
city_ibge_code,2807856
date,2818241
epidemiological_week,2818241
estimated_population,2807856
estimated_population_2019,2807856
is_last,2818241
is_repeated,2818241
last_available_confirmed,2818241
last_available_confirmed_per_100k_inhabitants,2793772


In [140]:
df_full_clean = df_full.dropna(subset=["last_available_confirmed_per_100k_inhabitants", "estimated_population"])

# 2) Cálculo de Métricas
Para cada município:
- Taxa de casos por 100 mil habitantes:
  - casos/habitantes * 100 000
- Taxa de óbitos por 100 mil habitantes:
  - obitos/habitantes * 100 000
- Taxa de mortalidade:
  - obitos/casos

In [141]:
# Atenção! Existem muitos municípios com o mesmo nome em estados diferentes:
counts = df_clean["city"].value_counts()
counts[counts>1]

,count
city,
São Domingos,5
Bom Jesus,5
Planalto,4
São Francisco,4
Vera Cruz,4
...,...
Petrolândia,2
Tabatinga,2
São Carlos,2


Portanto vamos usar o indentificador único do ibge.

In [142]:
df_resposta = df_clean[["city", "state", "city_ibge_code"]].copy()

df_resposta["taxa_casos_100k"] = df_clean["confirmed"] / df_clean["estimated_population"] * 10**5
df_resposta["taxa_obitos_100k"] = df_clean["deaths"] / df_clean["estimated_population"] * 10**5
df_resposta["taxa_mortalidade"] = df_clean["deaths"] / df_clean["confirmed"]

df_resposta

,city,state,city_ibge_code,taxa_casos_100k,taxa_obitos_100k,taxa_mortalidade
734,Amapá,AP,1600105.0,15500.163274,141.504300,0.009129
1320,Calçoene,AP,1600204.0,16380.682823,123.828056,0.007559
1901,Cutias,AP,1600212.0,14325.520406,114.735289,0.008009
2478,Ferreira Gomes,AP,1600238.0,19656.081336,87.862433,0.004470
3058,Itaubal,AP,1600253.0,9791.703756,53.409293,0.005455
...,...,...,...,...,...,...
2835558,Águas de São Pedro,SP,3500600.0,13092.871343,340.812269,0.026030
2836080,Álvares Florence,SP,3501202.0,17164.792981,438.716753,0.025559
2836553,Álvares Machado,SP,3501301.0,10028.802304,316.025282,0.031512
2837057,Álvaro de Carvalho,SP,3501400.0,5536.594615,170.648464,0.030822


# 3) Ordenação:
Implemente os algoritmos de ordenação abaixo e aplique-os para classificar os municípios por:
- Maior taxa de casos por 100 mil hab.
- Maior taxa de mortalidade.
- Menor taxa de mortalidade.

Compare:
- Merge Sort
- Quick Sort
- Função nativa da linguagem (ex: sort() do Python)

In [143]:
def merge(vL, vR, compare=lambda x,y : x<y): # auxiliary O(n) space
  out = []
  i = j = 0
  # inserting
  while i < len(vL) and j < len(vR):
    if compare(vL[i], vR[j]): # rev
      out.append(vL[i])
      i += 1
    else:
      out.append(vR[j])
      j += 1
  # extending
  while i < len(vL):
    out.append(vL[i])
    i += 1
  while j < len(vR):
    out.append(vR[j])
    j += 1

  return out

def mergesort(v, l=0, r=None, compare=lambda x,y : x<y):
  if r == None:
    r = len(v)-1
  # base case
  if l == r:
    return [v[l]]
  # divide T(n//2)
  m = (l+r)//2
  vL = mergesort(v, l, m, compare)
  vR = mergesort(v, m+1, r, compare)
  # conquer O(n)
  return merge(vL, vR, compare)

In [144]:
import random

def partition(v, l, r, compare=lambda x,y : x<y):
  pidx = random.randint(l, r)
  v[pidx], v[r] = v[r], v[pidx]  # move o pivô para o fim
  pivo = v[r]
  i = l
  for j in range(l,r):
    if compare(v[j], pivo):
      v[i], v[j] = v[j], v[i]
      i+=1
  v[i], v[r] = v[r], v[i]
  return i

def quicksort(v, l=0, r=None, compare=lambda x,y : x<y):  # inplace
  if r == None:
    r = len(v)-1
  if l < r:
    # divide (random)
    pidx = partition(v, l, r, compare)
    # conquer
    quicksort(v, l, pidx-1, compare)
    quicksort(v, pidx+1, r, compare)

Como o DataFrame é bem grande, eu utilizaria a forma ITERATIVA dos algoritmos para evitar estouro da pilha de recursão.

In [145]:
def merge(v, l1, r1, l2, r2, compare=lambda x,y : x<y):
  out = []
  i, j = l1, l2

  while i <= r1 and j <= r2:
    if compare(v[i], v[j]):
      out.append(v[i])
      i += 1
    else:
      out.append(v[j])
      j += 1

  while i <= r1:
    out.append(v[i])
    i += 1
  while j <= r2:
    out.append(v[j])
    j += 1

  for k in range(len(out)):
        v[l1 + k] = out[k]

def mergesort(v, l=0, r=None, compare=lambda x,y : x<y): # topdown iterative
  if r == None:
    r = len(v)-1
  n = len(v)
  if n <= 1:
    return v
  stack = [(0, n-1, False)]
  while stack:
    l, r, divided = stack.pop()
    if l < r:
      m = (l + r) // 2
      if not divided:
        # dividir
        stack.append((l, r, True)) # marcar como dividido
        stack.append((m+1, r, 0)) # direita
        stack.append((l, m, 0)) # esquerda
      else:
        merge(v, l, m, m+1, r, compare)

def quicksort(v, l=0, r=None, compare=lambda x,y : x<y):
  if r == None:
    r = len(v)-1
  stack = [(l,r)]
  while stack:
    l,r = stack.pop()
    if l < r:
      pidx = partition(v, l, r, compare)
      stack.append((l,pidx-1))
      stack.append((pidx+1,r))

O DataFrame original está ordenado por data. O que implica em uma relação de ordem entre numero de casos, taxa de mortalidade, etc.

Portanto, para ser justo com o algoritmo do Quicksort, o ideal seria realizar um SHUFFLE antes de comparar os métodos.

In [146]:
df_test = df_full_clean.sample(frac=1).reset_index(drop=True) # assim dou shuffle no dataframe!

Para ordenar os DataFrames apartir das colunas, precisaria salvar um array de índices ordenados.

In [147]:
import time

def sort_benchmark(df, column, sample_size, reverse=False):
    sample = df.sample(sample_size).reset_index(drop=True)
    data = sample[column]
    idx_m, idx_q, idx_s = list(range(sample_size)), list(range(sample_size)), list(range(sample_size))

    # comparador baseado na ordem
    cmp = (lambda i, j: data[i] >= data[j]) if reverse else (lambda i, j: data[i] <= data[j])

    # Mergesort
    start = time.time()
    mergesort(idx_m, compare=cmp)
    end = time.time()
    print(f"Mergesort: {end - start} segundos")

    # Quicksort
    start = time.time()
    quicksort(idx_q, compare=cmp)
    end = time.time()
    print(f"Quicksort: {end - start} segundos")

    # Timsort
    start = time.time()
    idx_s.sort(key=lambda i: data[i], reverse=reverse)
    end = time.time()
    print(f"Timsort: {end - start} segundos\n")

    # amostra e idxs ordenados
    return sample, idx_s


In [148]:
sort_benchmark(df_test, "last_available_confirmed_per_100k_inhabitants", 100, reverse=True)
sort_benchmark(df_test, "last_available_death_rate", 100, reverse=True)
sort_benchmark(df_test, "last_available_death_rate", 100)
print()

Mergesort: 0.002328634262084961 segundos
Quicksort: 0.002367734909057617 segundos
Timsort: 0.0002090930938720703 segundos

Mergesort: 0.0022041797637939453 segundos
Quicksort: 0.002341032028198242 segundos
Timsort: 0.00020503997802734375 segundos

Mergesort: 0.0032198429107666016 segundos
Quicksort: 0.002966642379760742 segundos
Timsort: 0.0002338886260986328 segundos




In [149]:
sort_benchmark(df_test, "last_available_confirmed_per_100k_inhabitants", 1000, reverse=True)
sort_benchmark(df_test, "last_available_death_rate", 1000, reverse=True)
sort_benchmark(df_test, "last_available_death_rate", 1000)
print()

Mergesort: 0.03577280044555664 segundos
Quicksort: 0.04522705078125 segundos
Timsort: 0.002158641815185547 segundos

Mergesort: 0.03410530090332031 segundos
Quicksort: 0.08338618278503418 segundos
Timsort: 0.0021598339080810547 segundos

Mergesort: 0.03611111640930176 segundos
Quicksort: 0.08668756484985352 segundos
Timsort: 0.002142667770385742 segundos




In [150]:
a, idx_a = sort_benchmark(df_test, "last_available_confirmed_per_100k_inhabitants", 10000, reverse=True)
b, idx_b = sort_benchmark(df_test, "last_available_death_rate", 10000, reverse=True)
c, idx_c = sort_benchmark(df_test, "last_available_death_rate", 10000)

Mergesort: 0.5107204914093018 segundos
Quicksort: 0.5981230735778809 segundos
Timsort: 0.024703264236450195 segundos

Mergesort: 0.4862947463989258 segundos
Quicksort: 6.477982759475708 segundos
Timsort: 0.02295970916748047 segundos

Mergesort: 0.512016773223877 segundos
Quicksort: 5.133244037628174 segundos
Timsort: 0.022600412368774414 segundos



Ordenar o dataframe (amostra) apartir dos indexes ordenados:

In [151]:
a_sorted = a.iloc[idx_a]
a_sorted

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
2321,Porteirão,5218052.0,2021-10-24,202143,3931.0,3881.0,False,False,1855,47189.01043,2021-10-24,0.0113,21,493,city,GO,0,0
3984,Porteirão,5218052.0,2021-09-19,202138,3931.0,3881.0,False,False,1854,47163.57161,2021-09-19,0.0119,22,458,city,GO,-1,0
3315,Severiano Melo,2413607.0,2021-11-03,202144,2088.0,2440.0,False,False,807,38649.42529,2021-11-03,0.0161,13,539,city,RN,0,0
4396,Severiano Melo,2413607.0,2021-07-27,202130,2088.0,2440.0,False,False,796,38122.60536,2021-07-27,0.0163,13,440,city,RN,1,0
9858,Paranaíta,5106299.0,2022-02-14,202207,11244.0,11225.0,False,False,4214,37477.76592,2022-02-14,0.0088,37,628,city,MT,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8022,Tatuí,3554003.0,2020-04-02,202014,122967.0,121766.0,False,False,1,0.81323,2020-04-02,0.0000,0,8,city,SP,0,0
7065,Santa Maria,4316907.0,2020-04-04,202014,283677.0,282123.0,False,False,2,0.70503,2020-04-04,0.0000,0,14,city,RS,0,0
3359,Pindamonhangaba,3538006.0,2020-04-10,202015,170132.0,168328.0,False,False,1,0.58778,2020-04-10,0.0000,0,4,city,SP,0,0
7635,Pelotas,4314407.0,2020-04-01,202014,343132.0,342405.0,False,False,2,0.58287,2020-04-01,0.0000,0,8,city,RS,0,0


In [152]:
b_sorted = b.iloc[idx_b]
b_sorted

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
1327,Janduís,2405207.0,2020-06-19,202025,5248.0,5268.0,False,False,1,19.05488,2020-06-19,1.0,1,33,city,RN,0,0
1645,Lucas do Rio Verde,5105259.0,2020-04-12,202016,67620.0,65534.0,False,False,1,1.47885,2020-04-12,1.0,1,11,city,MT,0,0
1727,Morro da Garça,3143609.0,2020-05-13,202020,2437.0,2462.0,False,False,1,41.03406,2020-05-13,1.0,1,21,city,MG,0,0
1886,Goiandira,5208509.0,2020-04-17,202016,5625.0,5600.0,False,False,1,17.77778,2020-04-17,1.0,1,10,city,GO,0,0
2490,Flores do Piauí,2203800.0,2020-07-23,202030,4462.0,4463.0,False,False,1,22.41147,2020-07-23,1.0,1,48,city,PI,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9965,Assunção,2501351.0,2020-09-02,202036,4029.0,3990.0,False,False,45,1116.90246,2020-09-02,0.0,0,101,city,PB,0,0
9967,Aurora do Tocantins,1702703.0,2020-12-09,202050,3783.0,3757.0,False,False,58,1533.17473,2020-12-09,0.0,0,216,city,TO,0,0
9975,Divisa Alegre,3122355.0,2020-07-02,202027,6868.0,6786.0,False,False,7,101.92196,2020-07-02,0.0,0,37,city,MG,0,0
9976,Janaúba,3135100.0,2020-05-24,202022,72018.0,71648.0,False,False,10,13.88542,2020-05-24,0.0,0,50,city,MG,0,0


In [153]:
c_sorted = c.iloc[idx_c]
c_sorted

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
1,Passabém,3147501.0,2020-09-05,202036,1633.0,1649.0,False,False,7,428.65891,2020-09-05,0.0,0,71,city,MG,0,0
10,Aroeiras,2501302.0,2020-05-21,202021,19116.0,19153.0,False,False,13,68.00586,2020-05-21,0.0,0,12,city,PB,1,0
11,Sagres,3544707.0,2020-09-04,202036,2430.0,2432.0,False,False,6,246.91358,2020-09-04,0.0,0,112,city,SP,0,0
14,Arapoema,1702307.0,2020-06-17,202025,6616.0,6643.0,False,False,3,45.34462,2020-06-17,0.0,0,36,city,TO,0,0
21,Jales,3524808.0,2020-05-11,202020,49201.0,49107.0,False,False,5,10.16240,2020-05-11,0.0,0,34,city,SP,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7521,Nova Esperança do Sul,4313037.0,2020-09-14,202038,5410.0,5352.0,False,False,1,18.48429,2020-09-14,1.0,1,6,city,RS,0,0
9201,Itacuruba,2607406.0,2020-06-07,202024,4966.0,4918.0,False,False,1,20.13693,2020-06-07,1.0,1,11,city,PE,0,0
9578,Senador Firmino,3165701.0,2020-06-18,202025,7858.0,7812.0,False,False,1,12.72588,2020-06-18,1.0,1,7,city,MG,0,0
9735,Coronel Macedo,3512605.0,2020-07-02,202027,4635.0,4681.0,False,False,2,43.14995,2020-07-02,1.0,2,35,city,SP,0,0


O Quicksort é um algoritmo de ordenação INSTÁVEL, altera a ordenação de elementos iguais, já o Timsort (python nativo) e o Mergesort são estáveis.

O Quicksort também é INPLACE, altera o array original sem precisar de um espaço auxiliar. Já o Mergesort e o Timsort (python nativo) utilizam um espaço O(n) auxiliar.

Em complexidade de tempo, o Mergesort é O(nlogn) em todos os casos.

O Quicksort depende de particionar a lista em sublistas, um algoritmo aleatório. Seu melhor caso é quando os pivôs escolhidos dividem a lista no meio sucessivamente, sendo O(n x logn). No pior caso, onde a lista diminui em apenas 1 elemento, é O(n x n). É provado que em média o Quicksort é O(n x logn).

O Timsort é um método hibrido em que utiliza Mergesort e aproveita o Insertion sort para sublistas pequenas. O(nlogn) no pior e caso médio, O(n) no melhor (caso já esteja ordenado).

# 4) Busca da Mediana - Quickselect

- A mediana das taxas de mortalidade entre os municípios.
- A mediana das taxas de casos por 100 mil habitantes.

Compare o desempenho do Quickselect com a abordagem tradicional
de ordenar e acessar o elemento central.

In [154]:
def median(v):
  v = sorted(v)
  #mergesort(v)
  n = len(v)
  if n%2 == 1:
    return v[n//2]
  else:
    return (v[n//2] + v[n//2 -1])/2

def quickmedian(v):
  n = len(v)
  if n%2 == 1:
    return quickselect(v, 0, n-1, n//2)
  else:
    return 0.5*(quickselect(v, 0, n-1, n//2 -1) + quickselect(v, 0, n-1, n//2))

def quickselect(v, l, r, k):
  p = partition(v,l,r)
  if p == k:
    return v[p]
  elif p < k:
    return quickselect(v, p+1, r, k)
  else:
    return quickselect(v, l, p-1, k)

Caso a base de dados ja esteja ordenada, encontrar a mediana é O(1).

Ordenar a lista usando, por exemplo o TimSort, acaba sendo O(nlogn).

Com o Quickselect, a complexidade é basicamente só o partition. No melhor caso (dividindo sempre ao meio) encontra a mediana em O(logn), no pior caso (diminuir somente em um a lista) é O(n).



In [155]:
v = list(df_full_clean["last_available_confirmed_per_100k_inhabitants"])

start = time.time()
median(v.copy())
end = time.time()
print(f"Median: {end-start}")

start = time.time()
quickmedian(v.copy())
end = time.time()
print(f"Quickmedian: {end-start}")

v = list(df_full_clean["last_available_death_rate"])

start = time.time()
median(v.copy())
end = time.time()
print(f"Median: {end-start}")

start = time.time()
quickmedian(v.copy())
end = time.time()
print(f"Quickmedian: {end-start}")


Median: 1.268620491027832
Quickmedian: 4.7825608253479
Median: 0.6722195148468018
Quickmedian: 31.678667068481445


# 5) Avaliação de desempenho
- Meça o tempo de execução de cada algoritmo para diferentes tamanho de entrada (e.g., 100, 1000, 10000 linhas).
- Compare os resultados com a complexidade esperada.


# 6) Relatório
- Gráficos de desempenho (tempo vs tamanho).
- Tabelas com os Top 10 municípios segundo cada critério.
- Valor da mediana das taxas encontradas e sua interpretação
- Análise crítica dos resultados (breve)